# Starter API Access Tunnel

In [4]:
# Dependencies
import datetime as dt
import requests
import pandas as pd
# from pathlib import Path
from pprint import pprint

import api_keys

In [6]:
fema_base = 'https://www.fema.gov/api/open/'
disaster_endpoint = 'v2/DisasterDeclarationsSummaries?format=json'
regions_endpoint = 'v2/FemaRegions?format=json'

disaster_API = fema_base + disaster_endpoint
regions_API = fema_base + regions_endpoint

In [7]:
fema_response = requests.get(disaster_API).json()

In [1]:
pprint(fema_response)

Pretty printing has been turned OFF


In [ ]:
geoapify_base = 'https://api.geoapify.com/v1/geocode/search?'
geoapify_params = {
    'apiKey' = api_keys.geoapify_key,
    
}